In [2]:
include("../src/ProcessSequence.jl")

Main.ProcessSequence

In [3]:
project_dir = dirname(@__DIR__)

using JSON: parse

project_json = parse(read(joinpath(project_dir, "project.json"), String))

sample_name = project_json["sample_name"]

output_dir = joinpath(project_dir, "output")

input_dir = joinpath(project_dir, "input")

sample_dir = joinpath(input_dir, sample_name)

reference_dir = joinpath(input_dir, "reference")

snpeff_jar = "/opt/snpeff/snpEff/snpEff.jar"

"/opt/snpeff/snpEff/snpEff.jar"

In [4]:
if !isdir(reference_dir)

    ProcessSequence.print_and_run_cmd(`unzip -o -d $input_dir $reference_dir.zip`)
    
end

In [6]:
if !isfile(snpeff_jar)
    
    throw("$snpeff is missing.")
    
end

In [5]:
ProcessSequence.check_program()

Checking program...
`which skewer`
/opt/conda/bin/skewer
`which fastqc`
/opt/conda/bin/fastqc
`which bgzip`
/opt/conda/bin/bgzip
`which tabix`
/opt/conda/bin/tabix
`which minimap2`
/opt/conda/bin/minimap2
`which samtools`
/opt/conda/bin/samtools
`which bcftools`
/opt/conda/bin/bcftools
`which kallisto`
/opt/conda/bin/kallisto
`bash -c 'source activate py2 && which configManta.py'`
/opt/conda/envs/py2/bin/configManta.py
`bash -c 'source activate py2 && which configureStrelkaGermlineWorkflow.py'`
/opt/conda/envs/py2/bin/configureStrelkaGermlineWorkflow.py
`bash -c 'source activate py2 && which configureStrelkaSomaticWorkflow.py'`
/opt/conda/envs/py2/bin/configureStrelkaSomaticWorkflow.py


In [6]:
process_dna_arguments = (
    joinpath(
        reference_dir,
        "GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz",
    ),
    joinpath(reference_dir, "chromosome.bed.gz"),
    joinpath(reference_dir, "chrn_n.tsv"),
    project_json["n_job"],
    project_json["gb_memory"],
    1,
    snpeff_jar,
)

("/home/jovyan/ProcessSequence.jl/input/reference/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz", "/home/jovyan/ProcessSequence.jl/input/reference/chromosome.bed.gz", "/home/jovyan/ProcessSequence.jl/input/reference/chrn_n.tsv", 8, 40, 1, "/opt/snpeff/snpEff/snpEff.jar")

In [7]:
if all((
    in(key, keys(project_json)) for key in ("germ_dna.1.fastq.gz", "germ_dna.2.fastq.gz")
))

    ProcessSequence.process_germ_dna(
        joinpath(project_dir, project_json["germ_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["germ_dna.2.fastq.gz"]),
        project_json["dna_is_targeted"],
        joinpath(output_dir, string("process_germ_dna_", sample_name)),
        process_dna_arguments...,
    )

end

(2020-12-10T18:10:59.124) Trimming sequence ...
`skewer --threads 8 -x AGATCGGAAGAGC --end-quality 20 --mode pe --compress --output /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1005/trim_sequence/germ --quiet /home/jovyan/ProcessSequence.jl/input/1005_cat/1005_R1.fastq.gz /home/jovyan/ProcessSequence.jl/input/1005_cat/1005_R2.fastq.gz`
.--. .-.
: .--': :.-.
`. `. : `'.' .--. .-..-..-. .--. .--.
_`, :: . `.' '_.': `; `; :' '_.': ..'
`.__.':_;:_;`.__.'`.__.__.'`.__.':_;
skewer v0.2.2 [April 4, 2016]
Parameters used:
-- 3' end adapter sequence (-x):	AGATCGGAAGAGC
-- maximum error ratio allowed (-r):	0.100
-- maximum indel error ratio allowed (-d):	0.030
-- end quality threshold (-q):		20
-- minimum read length allowed after trimming (-l):	18
-- file format (-f):		Sanger/Illumina 1.8+ FASTQ (auto detected)
-- number of concurrent threads (-t):	8
Thu Dec 10 18:10:59 2020 >> started

Thu Dec 10 21:03:34 2020 >> done (10367.530s)
300122445 read pairs processed; of these:
      765 

Started analysis of germ-trimmed-pair1.fastq.gz
Started analysis of germ-trimmed-pair2.fastq.gz
Approx 5% complete for germ-trimmed-pair1.fastq.gz
Approx 5% complete for germ-trimmed-pair2.fastq.gz
Approx 10% complete for germ-trimmed-pair1.fastq.gz
Approx 10% complete for germ-trimmed-pair2.fastq.gz
Approx 15% complete for germ-trimmed-pair1.fastq.gz
Approx 15% complete for germ-trimmed-pair2.fastq.gz
Approx 20% complete for germ-trimmed-pair1.fastq.gz
Approx 20% complete for germ-trimmed-pair2.fastq.gz
Approx 25% complete for germ-trimmed-pair1.fastq.gz
Approx 25% complete for germ-trimmed-pair2.fastq.gz
Approx 30% complete for germ-trimmed-pair1.fastq.gz
Approx 30% complete for germ-trimmed-pair2.fastq.gz
Approx 35% complete for germ-trimmed-pair1.fastq.gz
Approx 35% complete for germ-trimmed-pair2.fastq.gz
Approx 40% complete for germ-trimmed-pair1.fastq.gz
Approx 40% complete for germ-trimmed-pair2.fastq.gz
Approx 45% complete for germ-trimmed-pair1.fastq.gz
Approx 45% complete fo

Analysis complete for germ-trimmed-pair1.fastq.gz
Analysis complete for germ-trimmed-pair2.fastq.gz
(2020-12-10T22:01:55.45) Done in 58 minutes, 18 seconds, 760 milliseconds.
(2020-12-10T22:01:55.468) Aligning sequence ...
pipeline(pipeline(pipeline(pipeline(`minimap2 -x sr -t 8 -K 1G -R '@RG\tID:Germ\tSM:Germ' -a /home/jovyan/ProcessSequence.jl/input/reference/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.gz.mmi /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1005/trim_sequence/germ-trimmed-pair1.fastq.gz /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1005/trim_sequence/germ-trimmed-pair2.fastq.gz`, stdout=`samtools sort --threads 8 -n`), stdout=`samtools fixmate --threads 8 -m - -`), stdout=`samtools sort --threads 8`), stdout>Base.FileRedirect("/home/jovyan/ProcessSequence.jl/output/process_germ_dna_1005/align_sequence/germ.bam.tmp", false))


[WARNING] Indexing parameters (-k, -w or -H) overridden by parameters used in the prebuilt index.
[M::main::47.820*0.33] loaded/built the index for 195 target sequence(s)
[M::mm_mapopt_update::47.820*0.33] mid_occ = 1000
[M::mm_idx_stat] kmer size: 15; skip: 10; is_hpc: 0; #seq: 195
[M::mm_idx_stat::49.046*0.35] distinct minimizers: 100167746 (38.80% are singletons); average occurrences: 5.519; average spacing: 5.607
[M::worker_pipeline::206.547*5.79] mapped 6670422 sequences
[M::worker_pipeline::329.091*6.76] mapped 6670678 sequences
[M::worker_pipeline::514.938*7.27] mapped 6671156 sequences
[M::worker_pipeline::603.452*7.45] mapped 6670908 sequences
[M::worker_pipeline::730.625*7.60] mapped 6671036 sequences
[M::worker_pipeline::924.190*7.72] mapped 6671722 sequences
[M::worker_pipeline::1021.284*7.79] mapped 6671238 sequences
[M::worker_pipeline::1141.459*7.85] mapped 6670182 sequences
[M::worker_pipeline::1335.839*7.90] mapped 6670444 sequences
[M::worker_pipeline::1422.175*7.94] 

`samtools markdup --threads 8 -s /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1005/align_sequence/germ.bam.tmp /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1005/align_sequence/germ.bam`


READ 601637239 WRITTEN 601637239 
EXCLUDED 406463442 EXAMINED 195173797
PAIRED 194375742 SINGLE 798055
DULPICATE PAIR 45166053 DUPLICATE SINGLE 312875
DUPLICATE TOTAL 45478928


`samtools index -@ 8 /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1005/align_sequence/germ.bam`
pipeline(`samtools flagstat --threads 8 /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1005/align_sequence/germ.bam`, stdout>Base.FileRedirect("/home/jovyan/ProcessSequence.jl/output/process_germ_dna_1005/align_sequence/germ.bam.flagstat", false))
(2020-12-11T04:47:42.023) Done in 6 hours, 45 minutes, 46 seconds, 555 milliseconds.
(2020-12-11T04:47:42.407) Finding variant ...
`bash -c 'source activate py2 && configManta.py --referenceFasta /home/jovyan/ProcessSequence.jl/input/reference/GCA_000001405.15_GRCh38_no_alt_analysis_set.fna.bgz --callRegions /home/jovyan/ProcessSequence.jl/input/reference/chromosome.bed.gz --bam /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1005/align_sequence/germ.bam --outputContig --runDir /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1005/find_variant/manta && /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1005/f

00:00:00	SnpEff version SnpEff 5.0c (build 2020-11-25 14:23), by Pablo Cingolani
00:00:00	Command: 'ann'
00:00:00	Reading configuration file 'snpEff.config'. Genome: 'GRCh38.99'
00:00:00	Reading config file: /home/jovyan/ProcessSequence.jl/notebook/snpEff.config
00:00:00	Reading config file: /opt/snpeff/snpEff/snpEff.config
00:00:00	done
00:00:00	Reading database for genome version 'GRCh38.99' from file '/opt/snpeff/snpEff/./data/GRCh38.99/snpEffectPredictor.bin' (this might take a while)
00:00:22	done
00:00:22	Loading Motifs and PWMs
00:00:22	Building interval forest
00:00:27	done.
00:00:27	Genome stats :
#-----------------------------------------------
# Genome name                : 'Homo_sapiens'
# Genome version             : 'GRCh38.99'
# Genome ID                  : 'GRCh38.99[0]'
# Has protein coding info    : true
# Has Tr. Support Level info : true
# Genes                      : 60676
# Protein coding genes       : 20135
#-----------------------------------------------
# Trans

`tabix /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1005/find_variant/snpeff/snpeff.vcf.gz`
pipeline(pipeline(`bcftools view --threads 8 --include 'FILTER=="PASS"' /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1005/find_variant/snpeff/snpeff.vcf.gz`, stdout=`bgzip --threads 8 --stdout`), stdout>Base.FileRedirect("/home/jovyan/ProcessSequence.jl/output/process_germ_dna_1005/find_variant/pass.vcf.gz", false))
`tabix /home/jovyan/ProcessSequence.jl/output/process_germ_dna_1005/find_variant/pass.vcf.gz`
(2020-12-11T06:48:59.436) Done in 2 hours, 1 minute, 17 seconds, 29 milliseconds.


In [ ]:
if all((
    in(key, keys(project_json))
    for
    key in (
        "germ_dna.1.fastq.gz",
        "germ_dna.2.fastq.gz",
        "soma_dna.1.fastq.gz",
        "soma_dna.2.fastq.gz",
    )
))

    ProcessSequence.process_soma_dna(
        joinpath(project_dir, project_json["germ_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["germ_dna.2.fastq.gz"]),
        joinpath(project_dir, project_json["soma_dna.1.fastq.gz"]),
        joinpath(project_dir, project_json["soma_dna.2.fastq.gz"]),
        project_json["dna_is_targeted"],
        joinpath(output_dir, "process_soma_dna"),
        process_dna_arguments...,
    )

end

In [ ]:
if all((
    in(key, keys(project_json)) for key in ("soma_rna.1.fastq.gz", "soma_rna.2.fastq.gz")
))

    ProcessSequence.process_soma_rna(
        joinpath(project_dir, project_json["soma_rna.1.fastq.gz"]),
        joinpath(project_dir, project_json["soma_rna.2.fastq.gz"]),
        joinpath(output_dir, "process_soma_rna"),
        joinpath(reference_dir, "Homo_sapiens.GRCh38.cdna.all.fa.gz"),
        project_json["n_job"],
    )

end